In [1]:
import cv2
import os
import json
import unicodedata

# Türkçe karakterleri ASCII uyumlu hale getir
def normalize_filename(name):
    name = unicodedata.normalize('NFKD', name)
    return ''.join(c for c in name if not unicodedata.combining(c))

def process_video(video_path, label, output_folder, json_data):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Video açılırken hata oluştu: {video_path}")
        return
    
    frame_count = 0
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_filename = f"{label}_{frame_count}.jpg"
        frame_path = os.path.join(output_folder, frame_filename)

        try:
            cv2.imwrite(frame_path, frame)
            print(f"Görsel kaydedildi: {frame_path}")
        except Exception as e:
            print(f"Resim kaydedilirken hata oluştu: {e}")
            continue

        frames.append({
            "image_path": frame_path,
            "label": label
        })

        frame_count += 1

    cap.release()
    json_data[label] = frames
    print(f"{label} etiketli video işlendi, {frame_count} kare kaydedildi.")

def save_json(json_data, output_json_path):
    try:
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, ensure_ascii=False, indent=4)
        print(f"Veriler {output_json_path} dosyasına kaydedildi.")
    except Exception as e:
        print(f"JSON dosyasına yazılırken hata oluştu: {e}")

def process_videos(video_folder, output_folder, json_output_path):
    json_data = {}

    for video_name in os.listdir(video_folder):
        if video_name.endswith(".mp4"):
            label = os.path.splitext(video_name)[0]
            label_normalized = normalize_filename(label)  # Türkçe karakterleri sadeleştir

            video_path = os.path.join(video_folder, video_name)
            print(f"{video_name} işleniyor... (Etiket: {label_normalized})")
            process_video(video_path, label_normalized, output_folder, json_data)

    save_json(json_data, json_output_path)

# Ana çağrı
if __name__ == "__main__":
    video_folder = r"İşaretDiliVideo"
    output_folder = r"OutputImages"
    json_output_path = r"output_data.json"

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    process_videos(video_folder, output_folder, json_output_path)


Anadolu.mp4 işleniyor... (Etiket: Anadolu)
Görsel kaydedildi: OutputImages\Anadolu_0.jpg
Görsel kaydedildi: OutputImages\Anadolu_1.jpg
Görsel kaydedildi: OutputImages\Anadolu_2.jpg
Görsel kaydedildi: OutputImages\Anadolu_3.jpg
Görsel kaydedildi: OutputImages\Anadolu_4.jpg
Görsel kaydedildi: OutputImages\Anadolu_5.jpg
Görsel kaydedildi: OutputImages\Anadolu_6.jpg
Görsel kaydedildi: OutputImages\Anadolu_7.jpg
Görsel kaydedildi: OutputImages\Anadolu_8.jpg
Görsel kaydedildi: OutputImages\Anadolu_9.jpg
Görsel kaydedildi: OutputImages\Anadolu_10.jpg
Görsel kaydedildi: OutputImages\Anadolu_11.jpg
Görsel kaydedildi: OutputImages\Anadolu_12.jpg
Görsel kaydedildi: OutputImages\Anadolu_13.jpg
Görsel kaydedildi: OutputImages\Anadolu_14.jpg
Görsel kaydedildi: OutputImages\Anadolu_15.jpg
Görsel kaydedildi: OutputImages\Anadolu_16.jpg
Görsel kaydedildi: OutputImages\Anadolu_17.jpg
Görsel kaydedildi: OutputImages\Anadolu_18.jpg
Görsel kaydedildi: OutputImages\Anadolu_19.jpg
Görsel kaydedildi: OutputIm

In [2]:
# Gerekli kütüphaneler içe aktarılıyor
import os  # Dosya ve dizin işlemleri için
import json  # JSON dosyalarını okumak ve yazmak için
import cv2  # Görüntü işleme işlemleri için OpenCV
import numpy as np  # Sayısal işlemler ve diziler için NumPy
import torch  # PyTorch derin öğrenme kütüphanesi
import torch.nn as nn  # Sinir ağı modülleri için
import torch.optim as optim  # Optimizasyon algoritmaları için
from torch.utils.data import DataLoader, Dataset  # Veri yükleyici ve özel veri kümesi sınıfı
import speech_recognition as sr  # Ses tanıma işlemleri için
import onnxruntime as ort  # ONNX modelini çalıştırmak için
import typing  # Tip tanımlamaları için (örneğin Tuple)
import threading
import time

# === AnimeGAN Tanımı ===
# Anime stilinde görüntü oluşturmak için kullanılan AnimeGAN sınıfı tanımlanıyor
class AnimeGAN:
    # Sınıfın kurucu fonksiyonu (init), model yolu ve boyut küçültme oranı alıyor
    def __init__(self, model_path: str = '', downsize_ratio: float = 1.0):
        # Model dosyasının var olup olmadığı kontrol ediliyor
        if not os.path.exists(model_path):
            raise Exception(f"Model bulunamadı: {model_path}")
        
        # Görüntüyü küçültmek için kullanılacak oran atanıyor
        self.downsize_ratio = downsize_ratio
        
        # Modelin çalışacağı cihaz belirleniyor (GPU varsa GPU, yoksa CPU)
        providers = ['CUDAExecutionProvider'] if ort.get_device() == "GPU" else ['CPUExecutionProvider']
        
        # ONNX model oturumu başlatılıyor
        self.ort_sess = ort.InferenceSession(model_path, providers=providers)

    # Görüntü boyutlarını 32'nin katına yuvarlayan yardımcı fonksiyon
    def to_32s(self, x):
        return 256 if x < 256 else x - x % 32

    # Girdi görüntüsünü model için işleyen fonksiyon
    def process_frame(self, frame: np.ndarray, x32: bool = True) -> np.ndarray:
        h, w = frame.shape[:2]  # Yükseklik ve genişlik alınır
        if x32:
            frame = cv2.resize(frame, (self.to_32s(int(w * self.downsize_ratio)), self.to_32s(int(h * self.downsize_ratio))))
        frame = frame.astype(np.float32) / 127.5 - 1.0  # Görüntü verisi -1 ile 1 aralığına ölçeklenir
        return frame

    # Model çıktılarını işleyerek tekrar orijinal formata dönüştürür
    def post_process(self, frame: np.ndarray, wh: typing.Tuple[int, int]) -> np.ndarray:
        frame = (frame.squeeze() + 1.) / 2 * 255  # -1 ile 1 aralığından 0-255'e normalize edilir
        frame = frame.astype(np.uint8)  # Görüntü verisi uint8 formatına çevrilir
        frame = cv2.resize(frame, (wh[0], wh[1]))  # Görüntü orijinal boyuta getirilir
        return frame

    # Sınıfın çağrıldığında çalışacak ana fonksiyonu
    def __call__(self, frame: np.ndarray) -> np.ndarray:
        image = self.process_frame(frame)  # Girdi görüntüsü işlenir
        outputs = self.ort_sess.run(None, {self.ort_sess._inputs_meta[0].name: np.expand_dims(image, axis=0)})  # Model çalıştırılır
        return self.post_process(outputs[0], frame.shape[:2][::-1])  # Çıktı işlenip geri döndürülür

# === Veri ve Model Yolları ===
# JSON veri dosyasının ve resimlerin yolu tanımlanıyor
json_path = r"output_data.json"
image_folder = r"OutputImages"
animegan_model_path = r"animegan.onnx"  # ONNX model dosyasının yolu

# === Dataset Sınıfı ===
# İşaret dili veri kümesi için özel bir PyTorch Dataset sınıfı
class SignLanguageDataset(Dataset):
    # Kurucu fonksiyon; JSON dosyasını yükler ve verileri hazırlar
    def __init__(self, json_path, image_size=(64, 64)):
        with open(json_path, "r", encoding="utf-8") as f:
            self.data = json.load(f)  # JSON verisi yüklenir
        
        self.image_paths = []  # İşlenecek görüntülerin listesi
        self.labels = []  # Görüntülere karşılık gelen etiketler
        self.label_map = {word: idx for idx, word in enumerate(self.data.keys())}  # Kelimelere sayısal etiket ataması

        for word, frames in self.data.items():  # Her kelime ve ilgili kareleri döner
            for frame in frames:  # Her kareyi işler
                img_path = os.path.normpath(os.path.join(image_folder, os.path.basename(frame["image_path"])))  # Resim yolu oluşturulur
                if os.path.exists(img_path):  # Dosya mevcutsa
                    img = cv2.imread(img_path)  # Görüntü okunur
                    if img is not None:  # Görüntü başarıyla yüklendiyse
                        img = cv2.resize(img, image_size)  # Yeniden boyutlandırılır
                        img = img / 255.0  # 0-1 aralığına normalize edilir
                        self.image_paths.append(img)  # Görüntü listeye eklenir
                        self.labels.append(self.label_map[word])  # Etiket listeye eklenir
                    else:
                        print(f"Resim yüklenemedi: {img_path}")
                else:
                    print(f"Dosya yok: {img_path}")

        self.image_paths = np.array(self.image_paths)  # Görüntü listesi NumPy dizisine çevrilir
        self.labels = np.array(self.labels)  # Etiketler NumPy dizisine çevrilir

    def __len__(self):
        return len(self.image_paths)  # Veri kümesinin uzunluğu

    def __getitem__(self, idx):
        img = self.image_paths[idx]  # Belirtilen indeksdeki görüntü alınır
        label = self.labels[idx]  # Etiketi alınır
        img = np.transpose(img, (2, 0, 1))  # Kanal sırası PyTorch için (C, H, W) olacak şekilde düzenlenir
        return torch.tensor(img, dtype=torch.float32), torch.tensor(label, dtype=torch.long)  # Tensör olarak döndürülür

# === Model Tanımı ===
# CNN (Convolutional Neural Network) tabanlı model tanımlanıyor
class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()  # Üst sınıfın init fonksiyonu çağrılır
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)  # İlk konvolüsyon katmanı (3 kanal giriş, 32 filtre)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  # İkinci konvolüsyon katmanı
        self.pool = nn.MaxPool2d(2, 2)  # Havuzlama katmanı (2x2 boyutunda)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)  # Tam bağlantılı katman
        self.fc2 = nn.Linear(128, num_classes)  # Çıkış katmanı, sınıf sayısı kadar nöron
        self.dropout = nn.Dropout(0.5)  # Aşırı öğrenmeyi engellemek için dropout uygulanır

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # İlk konvolüsyon ve aktivasyon + havuzlama
        x = self.pool(torch.relu(self.conv2(x)))  # İkinci konvolüsyon ve aktivasyon + havuzlama
        x = x.view(-1, 64 * 16 * 16)  # Tensör düzleştirilir (flatten)
        x = torch.relu(self.fc1(x))  # Tam bağlantılı katman + aktivasyon
        x = self.dropout(x)  # Dropout uygulanır
        return self.fc2(x)  # Sonuç döndürülür

# === Eğitim ve Veri Yükleme ===
dataset = SignLanguageDataset(json_path)  # Veri kümesi oluşturulur
train_size = int(0.8 * len(dataset))  # %80 eğitim, %20 test verisi olarak ayrılır
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])  # Veri kümesi bölünür
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # Eğitim verisi için yükleyici
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)  # Test verisi için yükleyici

num_classes = len(dataset.label_map)  # Sınıf sayısı veri kümesinden alınır
model = CNNModel(num_classes)  # Model oluşturulur

criterion = nn.CrossEntropyLoss()  # Kayıp fonksiyonu (çok sınıflı sınıflandırma için)
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizasyon algoritması

# Eğitim döngüsü başlatılıyor
for epoch in range(10):  # 10 dönem boyunca model eğitilecek
    model.train()  # Eğitim moduna alınır
    total_loss = 0  # Toplam kayıp sıfırlanır
    for inputs, labels in train_loader:  # Tüm eğitim verileri döner
        optimizer.zero_grad()  # Gradyanlar sıfırlanır
        outputs = model(inputs)  # Model tahmin yapar
        loss = criterion(outputs, labels)  # Kayıp hesaplanır
        loss.backward()  # Geri yayılım yapılır
        optimizer.step()  # Ağırlıklar güncellenir
        total_loss += loss.item()  # Kayıp değeri toplanır
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")  # Epoch sonunda ortalama kayıp yazdırılır

torch.save(model.state_dict(), 'cnn_sign_language_model.pth')  # Model dosyası kaydedilir

# === Modeli Yükleme ===
model.load_state_dict(torch.load('cnn_sign_language_model.pth'))  # Eğitimli model geri yüklenir
model.eval()  # Değerlendirme moduna alınır




Dosya yok: OutputImages\Arasındaki_0.jpg
Dosya yok: OutputImages\Arasındaki_1.jpg
Dosya yok: OutputImages\Arasındaki_2.jpg
Dosya yok: OutputImages\Arasındaki_3.jpg
Dosya yok: OutputImages\Arasındaki_4.jpg
Dosya yok: OutputImages\Arasındaki_5.jpg
Dosya yok: OutputImages\Arasındaki_6.jpg
Dosya yok: OutputImages\Arasındaki_7.jpg
Dosya yok: OutputImages\Arasındaki_8.jpg
Dosya yok: OutputImages\Arasındaki_9.jpg
Dosya yok: OutputImages\Arasındaki_10.jpg
Dosya yok: OutputImages\Arasındaki_11.jpg
Dosya yok: OutputImages\Arasındaki_12.jpg
Dosya yok: OutputImages\Arasındaki_13.jpg
Dosya yok: OutputImages\Arasındaki_14.jpg
Dosya yok: OutputImages\Arasındaki_15.jpg
Dosya yok: OutputImages\Arasındaki_16.jpg
Dosya yok: OutputImages\Arasındaki_17.jpg
Dosya yok: OutputImages\Arasındaki_18.jpg
Dosya yok: OutputImages\Arasındaki_19.jpg
Dosya yok: OutputImages\Arasındaki_20.jpg
Dosya yok: OutputImages\Arasındaki_21.jpg
Dosya yok: OutputImages\Arasındaki_22.jpg
Dosya yok: OutputImages\Arasındaki_23.jpg
Do

CNNModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16384, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=41, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [ ]:
import cv2
import os
import json
import numpy as np

# AnimeGAN sınıfı tanımlı olmalı (önceden eklendiği gibi)
onnx_model_path = r'animegan.onnx'
animegan = AnimeGAN(model_path=onnx_model_path)  # AnimeGAN modeli yüklenir

# === JSON VERİSİ YÜKLE ===
with open("output_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)  # Bu satır eksikse hiçbir şey işlenmez!

# === İŞLEME FONKSİYONU ===
def preprocess_dataset(data, image_folder, output_folder="processed_images"):
    os.makedirs(output_folder, exist_ok=True)
    processed_data = {}

    for word, frames in data.items():
        processed_paths = []
        print(f"📦 Kelime işleniyor: {word}")
        for frame_info in frames:
            img_path = os.path.join(image_folder, os.path.basename(frame_info["image_path"]))
            print(f"📂 Dosya kontrol ediliyor: {img_path} - Var mı? {os.path.exists(img_path)}")
            if os.path.exists(img_path):
                img = cv2.imread(img_path)
                img = cv2.resize(img, (256, 256))
                anime_img = animegan(img)
                print("✨ AnimeGan filtresi uygulandı")
                save_path = os.path.join(output_folder, f"{word}_{len(processed_paths)}.jpg")
                cv2.imwrite(save_path, anime_img)
                processed_paths.append(save_path)

        processed_data[word] = processed_paths

    with open("processed_data.json", "w", encoding="utf-8") as f:
        json.dump(processed_data, f, ensure_ascii=False, indent=4)

    print("✅ Görseller başarıyla işlendi ve 'processed_data.json' dosyasına kaydedildi.")

# === FONKSİYON ÇAĞRISI ===
preprocess_dataset(data, "OutputImages")


In [8]:
import threading
import cv2
import time
import json
import speech_recognition as sr
import numpy as np

# === Türkçe Karakter Normalizasyon Fonksiyonu ===
def normalize_text(text):
    replacements = {
        "ç": "c", "ğ": "g", "ı": "i", "ö": "o", "ş": "s", "ü": "u",
        "Ç": "C", "Ğ": "G", "İ": "I", "Ö": "O", "Ş": "S", "Ü": "U"
    }
    for turkish_char, ascii_char in replacements.items():
        text = text.replace(turkish_char, ascii_char)
    return text

# JSON'dan işlenmiş veri yükleme
with open("processed_data.json", "r") as f:
    processed_data = json.load(f)

# Görüntüleri baştan belleğe al
cached_frames = {}
for word, paths in processed_data.items():
    cached_frames[word] = [cv2.imread(p) for p in paths]

def real_time_sign_language():
    words_queue = []
    stop_listening = threading.Event()

    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Kamera açılamadı.")
        return

    # Pencere ayarları
    screen_width = 3000
    screen_height = 1200
    cv2.namedWindow("İşaret Dili Animasyonu", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("İşaret Dili Animasyonu", screen_width, screen_height)
    cv2.moveWindow("İşaret Dili Animasyonu", 0, 0)

    def listen_microphone():
        recognizer = sr.Recognizer()
        mic = sr.Microphone()

        with mic as source:
            recognizer.adjust_for_ambient_noise(source)

        while not stop_listening.is_set():
            with mic as source:
                try:
                    print(" Konuşabilirsiniz...")
                    audio = recognizer.listen(source, timeout=3, phrase_time_limit=5)
                    text = recognizer.recognize_google(audio, language="tr-TR")
                    words = text.split()
                    if words:
                        normalized_words = [normalize_text(w).capitalize() for w in words]
                        words_queue.extend(normalized_words)
                        print(f" Algılandı: {' '.join(normalized_words)}")
                except sr.UnknownValueError:
                    print(" Anlaşılamadı, lütfen tekrar edin...")
                except sr.RequestError:
                    print(" API hatası, internet bağlantınızı kontrol edin.")
                except:
                    continue

    threading.Thread(target=listen_microphone, daemon=True).start()

    print(" Kamera başlatıldı. Konuştuğunuz kelimelerin işaret dilini gösterecek. Çıkmak için 'q' tuşuna basın...")

    current_word = ""
    frame_list = []
    last_word = ""
    word_start_time = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Aynalama kaldırıldı (flip yok)
        user_frame = cv2.resize(frame, (screen_width, screen_height))
        anime_frame = np.zeros((500, 500, 3), dtype=np.uint8)

        # Yeni kelime kuyruğu varsa ve animasyon yoksa
        if words_queue and not frame_list:
            current_word = words_queue.pop(0)
            if current_word in cached_frames:
                frame_list = cached_frames[current_word]
                word_start_time = time.time()
                last_word = current_word  # Son kelimeyi güncelle
            else:
                print(f" Veri bulunamadı: {current_word}")
                frame_list = []

        # Animasyon varsa zamanla göster
        if frame_list:
            frame_count = len(frame_list)
            elapsed_time = (time.time() - word_start_time) * 75
            index = int(elapsed_time)

            if index < frame_count:
                anime_frame = cv2.resize(frame_list[index], (500, 500))
            else:
                frame_list = []
        elif last_word in cached_frames:
            # Eğer yeni kelime algılanmadıysa, son kelimenin ilk karesi gösterilsin
            anime_frame = cv2.resize(cached_frames[last_word][0], (500, 500))

        # Anime frame'i sağ alt köşeye yerleştir
        x_offset = screen_width - 500 - 20
        y_offset = screen_height - 500 - 20
        user_frame[y_offset:y_offset + 500, x_offset:x_offset + 500] = anime_frame

        cv2.imshow("İşaret Dili Animasyonu", user_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_listening.set()
            break

    cap.release()
    cv2.destroyAllWindows()
    print(" Sistem durduruldu.")


In [27]:
# Canlı sistem başlat
real_time_sign_language()

 Kamera başlatıldı. Konuştuğunuz kelimelerin işaret dilini gösterecek. Çıkmak için 'q' tuşuna basın...
 Konuşabilirsiniz...
 Konuşabilirsiniz...
 Konuşabilirsiniz...
 Konuşabilirsiniz...
 Konuşabilirsiniz...
 Algılandı: Merhaba
 Konuşabilirsiniz...
 Algılandı: Ben Universite
 Konuşabilirsiniz...
 Algılandı: Ogrencisiyim
 Konuşabilirsiniz...
 Algılandı: Bugun Sabah Baslayan Trafik
 Konuşabilirsiniz...
 Algılandı: Anadolu Ve Avrupa Yakasi Arasindaki Gecislerde
 Konuşabilirsiniz...
 Konuşabilirsiniz...
 Algılandı: Uzun
 Konuşabilirsiniz...
 Algılandı: Kuyruklari Olusturdu
 Konuşabilirsiniz...
 Veri bulunamadı: Olusturdu
 Konuşabilirsiniz...
 Anlaşılamadı, lütfen tekrar edin...
 Konuşabilirsiniz...
 Algılandı: Olustu
 Konuşabilirsiniz...
 Algılandı: Belediyeden Yapilan Aciklamada
 Konuşabilirsiniz...
 Algılandı: Suruculer Uyarildi
 Konuşabilirsiniz...
 Sistem durduruldu.
